In [1]:
import pyspark.sql.functions as f
from pyspark.sql import Window
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.getOrCreate()
w = Window().orderBy(f.lit('A'))

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/06/19 23:46:33 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/06/19 23:46:36 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
#Leitura de CSV
df0 = spark.read.csv('work/Lista Pedidos.csv',sep=';',header='false');

In [4]:
df1 = df0.select(df0._c0.cast("integer").alias('CODIGO'),\
                        df0._c1.cast("string").alias('ORIGEM'),\
                        df0._c2.cast("string").alias('TIPO'),\
                        df0._c3.cast("integer").alias('QTITEM'),\
                        f.regexp_replace(f.regexp_replace(df0._c4,'[R$]',''), ',', '.').cast('double').alias('VLFINAL'),\
                        df0._c5.cast("string").alias('STATUS'),\
                        f.substring(df0._c6,1,2).cast("int").alias('DIA'),\
                        f.substring(df0._c6,4,2).cast("int").alias('MES'),\
                        f.substring(df0._c6,7,4).cast("int").alias('ANO'),\
                        f.substring(df0._c6,12,2).cast("int").alias('HORA'),\
                        f.substring(df0._c6,15,2).cast("int").alias('MINUTO'),\
                        df0._c7.cast("string").alias('TEMPO'),\
                        df0._c8.cast("string").alias('UNIDADE'),\
                        df0._c9.cast("string").alias('EXCLUSAO'))
print(f'Lista de Pedidos Formatados : {df1.count()}')

Lista de Pedidos Formatados : 11862


In [5]:
df2 = df1.distinct()

In [6]:
df3 = df2.dropna(how='all', subset='CODIGO')
print(f'Lista de Pedidos : {df3.count()}')

Lista de Pedidos : 11860


In [7]:
df_pedidos = df3.select(f.row_number().over(w).alias('ID'),'*')

In [8]:
df_pedidos.write.format("parquet").mode("overwrite").parquet('work/listapedidos')

22/06/19 23:47:20 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/06/19 23:47:20 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/06/19 23:47:24 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/06/19 23:47:25 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


In [9]:
dfMelhorDia = df_pedidos.groupBy("MES","DIA").agg(f.format_number(f.sum("VLFINAL"),2).alias("total"),\
                             f.format_number(f.avg("VLFINAL"),2).alias("media"),\
                             f.format_number(f.min("VLFINAL"),2).alias("menor"),\
                             f.format_number(f.max("VLFINAL"),2).alias("maior")).\
    orderBy(["MES","total"],ascending=[0, 0])

In [10]:
dfMelhorDia.show()


+---+---+--------+-----+-----+------+
|MES|DIA|   total|media|menor| maior|
+---+---+--------+-----+-----+------+
|  3| 19|4,445.87|24.70| 3.00| 83.00|
|  3| 12|4,290.17|26.16| 4.00|104.00|
|  3| 18|3,947.01|23.92| 4.00| 90.00|
|  3| 25|3,895.00|24.97| 0.00|133.00|
|  3| 11|3,814.50|23.40| 4.00| 85.99|
|  3| 21|3,786.66|20.36| 4.00| 79.50|
|  3|  2|3,753.67|24.53| 3.00| 77.50|
|  3| 22|3,540.71|21.72| 3.00| 67.50|
|  3| 24|3,478.49|19.99| 3.75| 55.00|
|  3|  7|3,472.14|24.45| 4.00|108.00|
|  3| 10|3,462.20|22.93| 4.00| 77.99|
|  3| 31|3,433.21|21.59| 0.00| 89.99|
|  3| 27|3,377.55|33.11| 4.00|138.48|
|  3| 28|3,245.98|22.54| 4.00| 66.00|
|  3| 15|3,234.56|21.86| 3.00| 88.99|
|  3|  5|3,219.76|22.84| 4.00| 84.00|
|  3|  8|3,178.56|22.38| 3.00|122.99|
|  3| 30|3,119.19|22.77| 4.00| 79.99|
|  3| 17|3,108.72|22.05| 4.00| 64.00|
|  3|  3|3,091.66|22.08| 4.00|115.99|
+---+---+--------+-----+-----+------+
only showing top 20 rows



In [21]:
df_pedidos.filter(f.col("CODIGO")==146979).show()

22/06/20 01:27:34 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/06/20 01:27:34 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/06/20 01:27:34 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/06/20 01:27:35 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+---+------+------+----+------+-------+------+---+---+---+----+------+-----+-------+--------+
| ID|CODIGO|ORIGEM|TIPO|QTITEM|VLFINAL|STATUS|DIA|MES|ANO|HORA|MINUTO|TEMPO|UNIDADE|EXCLUSAO|
+---+------+------+----+------+-------+------+---+---+---+----+------+-----+-------+--------+
+---+------+------+----+------+-------+------+---+---+---+----+------+-----+-------+--------+

